In [1]:
import tensorflow as tf
import numpy as np
import argparse
import os
import random
import time
import collections

type = "poetrySong"                   # dataset to use, shijing, songci, etc
trainPoems = "./dataset/" + type + "/" + type + ".txt" # training file location
checkpointsPath = "./checkpoints/" + type # checkpoints location

In [2]:
isEvaluate=False
poems = []
file = open(trainPoems, "r")
for line in file:  #every line is a poem
    title, author, poem = line.strip().split("::")  #get title and poem
    poem = poem.replace(' ','')
    if len(poem) < 10 or len(poem) > 512:  #filter poem
        continue
    if '_' in poem or '《' in poem or '[' in poem or '(' in poem or '（' in poem:
        continue
    poem = '[' + poem + ']' #add start and end signs
    poems.append(poem)
    #print(title, author, poem)

#counting words
wordFreq = collections.Counter()
for poem in poems:
    wordFreq.update(poem)
# print(wordFreq)

# erase words which are not common
#--------------------bug-------------------------
# word num less than original num, which causes nan value in loss function
# erase = []
# for key in wordFreq:
#     if wordFreq[key] < 2:
#         erase.append(key)
# for key in erase:
#     del wordFreq[key]

wordFreq[" "] = -1
wordPairs = sorted(wordFreq.items(), key = lambda x: -x[1])
words, freq = zip(*wordPairs)
wordNum = len(words)

wordToID = dict(zip(words, range(wordNum))) #word to ID
poemsVector = [([wordToID[word] for word in poem]) for poem in poems] # poem to vector
if isEvaluate: #evaluating need divide dataset into test set and train set
    trainVector = poemsVector[:int(len(poemsVector) * trainRatio)]
    testVector = poemsVector[int(len(poemsVector) * trainRatio):]
else:
    trainVector = poemsVector
    testVector = []
print("訓練樣本總數： %d" % len(trainVector))
print("測試樣本總數： %d" % len(testVector))

訓練樣本總數： 252478
測試樣本總數： 0


In [3]:
 def buildModel(wordNum, gtX, hidden_units = 128, layers = 2):
        """build rnn"""
        with tf.variable_scope("embedding"): #embedding
            embedding = tf.get_variable("embedding", [wordNum, hidden_units], dtype = tf.float32)
            inputbatch = tf.nn.embedding_lookup(embedding, gtX)

        basicCell = tf.contrib.rnn.BasicLSTMCell(hidden_units, state_is_tuple = True)
        stackCell = tf.contrib.rnn.MultiRNNCell([basicCell] * layers)
        initState = stackCell.zero_state(np.shape(gtX)[0], tf.float32)
        outputs, finalState = tf.nn.dynamic_rnn(stackCell, inputbatch, initial_state = initState)
        outputs = tf.reshape(outputs, [-1, hidden_units])

        with tf.variable_scope("softmax"):
            w = tf.get_variable("w", [hidden_units, wordNum])
            b = tf.get_variable("b", [wordNum])
            logits = tf.matmul(outputs, w) + b

        probs = tf.nn.softmax(logits)
        return logits, probs, stackCell, initState, finalState

In [4]:
def probsToWord(weights, words):
        """probs to word"""
        prefixSum = np.cumsum(weights) #prefix sum
        ratio = np.random.rand(1)
        index = np.searchsorted(prefixSum, ratio * prefixSum[-1]) # large margin has high possibility to be sampled
        return words[index[0]]

In [5]:
def testHead(characters):
        """write head poem"""
        print("genrating...")
        gtX = tf.placeholder(tf.int32, shape=[1, None])  # input
        logits, probs, stackCell, initState, finalState = buildModel(wordNum, gtX)
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            saver = tf.train.Saver()
            checkPoint = tf.train.get_checkpoint_state(checkpointsPath)
            # if have checkPoint, restore checkPoint
            if checkPoint and checkPoint.model_checkpoint_path:
                saver.restore(sess, checkPoint.model_checkpoint_path)
                print("restored %s" % checkPoint.model_checkpoint_path)
            else:
                print("no checkpoint found!")
                exit(1)
            flag = 1
            endSign = {-1: "，", 1: "。"}
            poem = ''
            state = sess.run(stackCell.zero_state(1, tf.float32))
            x = np.array([[wordToID['[']]])
            probs1, state = sess.run([probs, finalState], feed_dict={gtX: x, initState: state})
            for word in characters:
                if wordToID.get(word) == None:
                    print("胖虎不认识这个字，你真是文化人！")
                    exit(0)
                flag = -flag
                while word not in [']', '，', '。', ' ', '？', '！']:
                    poem += word
                    x = np.array([[wordToID[word]]])
                    probs2, state = sess.run([probs, finalState], feed_dict={gtX: x, initState: state})
                    word = probsToWord(probs2, words)

                poem += endSign[flag]
                # keep the context, state must be updated
                if endSign[flag] == '。':
                    probs2, state = sess.run([probs, finalState],
                                             feed_dict={gtX: np.array([[wordToID["。"]]]), initState: state})
                    poem += '\n'
                else:
                    probs2, state = sess.run([probs, finalState],
                                             feed_dict={gtX: np.array([[wordToID["，"]]]), initState: state})

            print(characters)
            print(poem)
            return poem

In [6]:
testHead("你爱我火")

genrating...
INFO:tensorflow:Restoring parameters from ./checkpoints/poetrySong/poetrySong-158999
restored ./checkpoints/poetrySong/poetrySong-158999
你爱我火
你至今历拂山前，爱我烦公不此脾。
我愿芳香故怜客，火盘陈隶瘦桓黄。



'你至今历拂山前，爱我烦公不此脾。\n我愿芳香故怜客，火盘陈隶瘦桓黄。\n'